# Ct value solver for Rotor-Gene Q
#### This script computes Ct values by retrieving raw excel files that were generated by Rotor-Gene Q and then converting them to the pandas dataframe. The computed Ct values are saved in an excel file "output.xlsx".

In [1]:
import pandas as pd
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
%matplotlib inline

/Users/kkim/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


#### A Ct value is computed for a given value of threshold (thresh). If there is no solution for the Ct value, returns np.nan.

In [2]:
def find_ct(x, y, thresh):
    x = np.array(x)
    y = np.array(y)
    tck = interpolate.splrep(x, y, s=0)
    xnew = np.arange(x.min(), x.max(), (x.max()-x.min())/50.)
    ynew = interpolate.splev(xnew, tck, der=0)
    tck_thresh = interpolate.splrep(x,y-thresh)
    try:  
        return interpolate.sproot(tck_thresh)[0]
    except:
        return np.nan

#### If you want to exclude multiple wells, e.g. 25, 30, 33, and 40, you can use a list like [25, 30, 33, 40]. Even though you want to remove a single well, a list needs to be used like [25], not 25.

In [3]:
def generate_df(file_name, threshold, well_number_excluded=[]):
    
    table = pd.read_excel(file_name)
    max_sample_num = len(table.columns)/3
    table = table.rename(columns={'Text': 'Text.0', 'X':'X.0', 'Y':'Y.0'})
    for i in reversed(range(0,max_sample_num)):
        table = table.rename(columns = {('Text.'+str(i)):('Text.'+str(i+1)), 
                             ('X.'+str(i)):('X.'+str(i+1)), 
                             ('Y.'+str(i)):('Y.'+str(i+1))})

    for i in well_number_excluded:
        str1 = 'Text.'+str(i)
        str2 = 'X.'+str(i)
        str3 = 'Y.'+str(i)
        table = table.drop(str1, 1)
        table = table.drop(str2, 1)
        table = table.drop(str3, 1)
    xs = np.array(table.columns).reshape(-1, 3).T[1]
    ys = np.array(table.columns).reshape(-1, 3).T[2]
    # print xs, ys

    well_number_included = range(1, max_sample_num+1)
    well_number_included = [i for i in well_number_included if i not in well_number_excluded]

    wells = []
    samples = []
    for i in well_number_included:
        well, sample  = table['Text.' + str(i)][0].split(':')
        wells.append(well)
        samples.append(sample)
    
    cts = []
    threshold = 2
    for i in range(0,len(xs)):
        ct = find_ct(table[xs[i]], table[ys[i]], threshold)
        cts.append(ct)
    
    df_cts = pd.DataFrame()
    df_cts['ct'] = cts
    well_numbers = [xs[i].split('.')[1] for i in range(0, len(xs))]

    df_cts['well number'] = map(int, wells)
    df_cts['sample'] = samples
    df_cts['file_name'] = file_name 
    return df_cts

#### sample format: read the file "#2 amplification.xls" without excluding any wells, and then append to the current dataframe df. 
df = df.append(generate_df('data/#2 amplification.xls', []))
#### When excluding 25 and 30 and 50.
df = df.append(generate_df('data/#3 amplification.xls', [25, 30, 50]))

In [4]:
df = pd.DataFrame()
df = df.append(generate_df('data/#1 amplification.xls', threshold = 2))
df = df.append(generate_df('data/#2 amplification.xls', threshold = 2))
df = df.append(generate_df('data/#3 amplification.xls', threshold = 2))
df = df.append(generate_df('data/#4 amplification.xls', threshold = 2))
df = df.append(generate_df('data/#5 amplification.xls', threshold = 2))
df = df.append(generate_df('data/#6 amplification.xls', threshold = 2))
df = df.append(generate_df('data/#7 amplification.xls', threshold = 2))
df = df.append(generate_df('data/#8 amplification.xls', threshold = 2))

print df

*** No CODEPAGE record, no encoding_override: will use 'ascii'
*** No CODEPAGE record, no encoding_override: will use 'ascii'
*** No CODEPAGE record, no encoding_override: will use 'ascii'
*** No CODEPAGE record, no encoding_override: will use 'ascii'
*** No CODEPAGE record, no encoding_override: will use 'ascii'
*** No CODEPAGE record, no encoding_override: will use 'ascii'
*** No CODEPAGE record, no encoding_override: will use 'ascii'
*** No CODEPAGE record, no encoding_override: will use 'ascii'
           ct  well number            sample                  file_name
0   22.366810            1    KK61_1_D01-A-A  data/#1 amplification.xls
1   22.874093            2    KK61_1_D01-A-A  data/#1 amplification.xls
2   22.595229            3    KK61_1_D01-A-A  data/#1 amplification.xls
3   21.947214            4    KK61_1_D01-A+A  data/#1 amplification.xls
4   21.839458            5    KK61_1_D01-A+A  data/#1 amplification.xls
5   21.900820            6    KK61_1_D01-A+A  data/#1 amplificat

#### The dataframe df is saved as an excel file 

In [5]:
writer = pd.ExcelWriter('output.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()